In [2]:
import os
import numpy as np
import mne
from hfd import hfd  # Higuchi’s Fractal Dimension function


def extracting_hfd_features(directory, freq_bands, num_participants=None, num_seconds=10, sampling_rate=250):
    """
    Extract HFD features per subject across all specified frequency bands,
    and save per-subject arrays of shape (n_segments, n_channels, n_bands),
    along with metadata: num_samples, num_channels, num_freqs (bands).
    """
    # Get list of .fif files
    file_names = sorted([f for f in os.listdir(directory) if f.endswith('.fif')])
    total_files = len(file_names)
    num_participants = total_files if num_participants is None else min(num_participants, total_files)

    # Label based on folder name
    # Assuming 'control' is part of the control group folder name
    label = 0 if 'control' in directory else 1
    print(f"Directory: {directory}  Label: {label}")

    # Prepare output directory
    base = os.path.basename(os.path.normpath(directory)).split('_')[0]
    out_dir = f"./{base}_hfd_features_new_MR"
    os.makedirs(out_dir, exist_ok=True)

    # Number of bands
    band_names = list(freq_bands.keys())
    n_bands = len(band_names)

    for i in range(num_participants):
        subj_file = file_names[i]
        subj_id = os.path.splitext(subj_file)[0]
        print(f"Processing subject: {subj_id}")

        # Read epochs once
        raw_epochs = mne.read_epochs(os.path.join(directory, subj_file), verbose=False)

        # Filter per band and store data
        filtered_data = {}
        for band in band_names:
            fmin, fmax = freq_bands[band]
            epochs_band = raw_epochs.copy().filter(fmin, fmax, verbose=False)
            filtered_data[band] = {
                'closed': epochs_band['Closed Eyes'].get_data(),  # (n_epochs, n_ch, n_times)
                'open':   epochs_band['Open Eyes'].get_data()
            }

        # Determine segmentation parameters
        # Assume each band-filtered signal has same shape
        n_epochs, n_channels, n_times = filtered_data[band_names[0]]['closed'].shape
        step = num_seconds * sampling_rate

        # Storage lists
        closed_hfd_feats = []  # list of (n_channels, n_bands)
        open_hfd_feats   = []
        labels = []

        # Iterate epochs and segments
        for epoch_ix in range(n_epochs):
            for start in range(0, n_times, step):
                end = start + step
                if end > n_times:
                    break
                # HFD matrix for this segment: channels x bands
                closed_seg_feats = []
                open_seg_feats   = []

                for band in band_names:
                    closed_seg = filtered_data[band]['closed'][epoch_ix, :, start:end]
                    open_seg   = filtered_data[band]['open'][epoch_ix, :, start:end]

                    # per-channel HFD
                    closed_vals = np.array([hfd(closed_seg[ch], kmax=None) for ch in range(n_channels)])
                    open_vals   = np.array([hfd(open_seg[ch],   kmax=None) for ch in range(n_channels)])

                    closed_seg_feats.append(closed_vals)
                    open_seg_feats.append(open_vals)

                # stack to (n_channels, n_bands)
                closed_hfd_feats.append(np.stack(closed_seg_feats, axis=1))
                open_hfd_feats.append(np.stack(open_seg_feats, axis=1))
                labels.append(label)

        # Convert lists to arrays
        closed_hfd_feats = np.array(closed_hfd_feats)  # (n_segments, n_channels, n_bands)
        open_hfd_feats   = np.array(open_hfd_feats)
        labels = np.array(labels)
        print("labels:", labels)
        # Metadata
        num_samples  = closed_hfd_feats.shape[0]
        num_channels = closed_hfd_feats.shape[1]
        num_freqs    = closed_hfd_feats.shape[2]
        print(f"  HFD array shape: {closed_hfd_feats.shape}")

        # Save per-subject
        npz_path = os.path.join(out_dir, f"{subj_id}_hfd.npz")
        np.savez(
            npz_path,
            hfd_closed=closed_hfd_feats,
            hfd_open=open_hfd_feats,
            labels=labels,
            num_samples=num_samples,
            num_channels=num_channels,
            num_freqs=num_freqs
        )
        print(f"  Saved HFD for {subj_id} → samples={num_samples}, channels={num_channels}, freqs={num_freqs}")

# Example usage:
# freq_bands = {
#     'Delta': (0.5, 4), 'Theta': (4, 8), 'Alpha': (8, 12), 'Beta': (12, 30), 'Gamma': (30, 45)
# }
# extracting_hfd_features("asmr_epochs/", freq_bands, num_participants=None, num_seconds=10, sampling_rate=250)

# Run on ASMR
freq_bands = {
    'Delta': (1, 4), 'Theta': (4, 8), 'Alpha': (8, 12), 'Beta': (12, 30), 'Gamma': (30, 45)
}
extracting_hfd_features(
    "./control_epochs/",
    freq_bands,
    num_participants=None,
    num_seconds=10,
    sampling_rate=250
)


Directory: ./control_epochs/  Label: 0
Processing subject: 1-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  HFD array shape: (18, 31, 5)
  Saved HFD for 1-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 10-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  HFD array shape: (18, 31, 5)
  Saved HFD for 10-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 11-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  HFD array shape: (18, 31, 5)
  Saved HFD for 11-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 12-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  HFD array shape: (18, 31, 5)
  Saved HFD for 12-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 13-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  HFD array shape: (18, 31, 5)
  Saved HFD for 13-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 14-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  H

In [8]:
import numpy as np

data = np.load("/home/s.dharia-ra/Shyamal/major_revisions_ASMR/control_hfd_features_new_MR/1-epochs-epo_hfd.npz")
data.keys()

features = data["hfd_open"]
features.shape

(18, 31, 5)

# PSD 

In [4]:
import os
import numpy as np
import mne
from scipy import signal


def extracting_psd_features(directory, freq_bands, num_participants=None, num_seconds=10, sampling_rate=250):
    """
    Extract PSD features per subject across all specified frequency bands,
    and save per-subject arrays of shape (n_segments, n_channels, n_bands),
    along with metadata: num_samples, num_channels, num_freqs (bands).
    """
    # Get list of .fif files
    file_names = sorted([f for f in os.listdir(directory) if f.endswith('.fif')])
    total_files = len(file_names)
    num_participants = total_files if num_participants is None else min(num_participants, total_files)

    # Label based on folder name
    # Assuming 'control' is part of the control group folder name
    label = 0 if 'control' in directory else 1
    print(f"Directory: {directory}  Label: {label}")

    # Prepare output directory
    base = os.path.basename(os.path.normpath(directory)).split('_')[0]
    out_dir = f"./{base}_psd_features_new_MR"
    os.makedirs(out_dir, exist_ok=True)

    # Number of bands
    band_names = list(freq_bands.keys())
    n_bands = len(band_names)

    for i in range(num_participants):
        subj_file = file_names[i]
        subj_id = os.path.splitext(subj_file)[0]
        print(f"Processing subject: {subj_id}")

        # Read epochs once
        raw_epochs = mne.read_epochs(os.path.join(directory, subj_file), verbose=False)

        # Get data for both conditions
        closed_data = raw_epochs['Closed Eyes'].get_data()  # (n_epochs, n_ch, n_times)
        open_data = raw_epochs['Open Eyes'].get_data()

        # Determine segmentation parameters
        n_epochs, n_channels, n_times = closed_data.shape
        step = num_seconds * sampling_rate

        # Storage lists
        closed_psd_feats = []  # list of (n_channels, n_bands)
        open_psd_feats = []
        labels = []

        # Iterate epochs and segments
        for epoch_ix in range(n_epochs):
            for start in range(0, n_times, step):
                end = start + step
                if end > n_times:
                    break
                
                # Extract segments for this time window
                closed_seg = closed_data[epoch_ix, :, start:end]  # (n_channels, n_times_seg)
                open_seg = open_data[epoch_ix, :, start:end]

                # PSD matrix for this segment: channels x bands
                closed_seg_feats = []
                open_seg_feats = []

                # Calculate PSD for each channel
                for ch in range(n_channels):
                    # Compute PSD using Welch's method
                    freqs_closed, psd_closed = signal.welch(
                        closed_seg[ch], 
                        fs=sampling_rate, 
                        nperseg=min(256, len(closed_seg[ch])),
                        noverlap=None
                    )
                    freqs_open, psd_open = signal.welch(
                        open_seg[ch], 
                        fs=sampling_rate, 
                        nperseg=min(256, len(open_seg[ch])),
                        noverlap=None
                    )

                    # Extract power in each frequency band
                    closed_band_powers = []
                    open_band_powers = []
                    
                    for band in band_names:
                        fmin, fmax = freq_bands[band]
                        
                        # Find frequency indices for this band
                        freq_mask_closed = (freqs_closed >= fmin) & (freqs_closed <= fmax)
                        freq_mask_open = (freqs_open >= fmin) & (freqs_open <= fmax)
                        
                        # Calculate mean power in this band (or sum)
                        band_power_closed = np.mean(psd_closed[freq_mask_closed])
                        band_power_open = np.mean(psd_open[freq_mask_open])
                        
                        closed_band_powers.append(band_power_closed)
                        open_band_powers.append(band_power_open)
                    
                    closed_seg_feats.append(closed_band_powers)
                    open_seg_feats.append(open_band_powers)

                # Convert to arrays and transpose to get (n_channels, n_bands)
                closed_psd_feats.append(np.array(closed_seg_feats))
                open_psd_feats.append(np.array(open_seg_feats))
                labels.append(label)

        # Convert lists to arrays
        closed_psd_feats = np.array(closed_psd_feats)  # (n_segments, n_channels, n_bands)
        open_psd_feats = np.array(open_psd_feats)
        labels = np.array(labels)
        print("labels:", labels)
        
        # Metadata
        num_samples = closed_psd_feats.shape[0]
        num_channels = closed_psd_feats.shape[1]
        num_freqs = closed_psd_feats.shape[2]
        print(f"  PSD array shape: {closed_psd_feats.shape}")

        # Save per-subject
        npz_path = os.path.join(out_dir, f"{subj_id}_psd.npz")
        np.savez(
            npz_path,
            psd_closed=closed_psd_feats,
            psd_open=open_psd_feats,
            labels=labels,
            num_samples=num_samples,
            num_channels=num_channels,
            num_freqs=num_freqs
        )
        print(f"  Saved PSD for {subj_id} → samples={num_samples}, channels={num_channels}, freqs={num_freqs}")


# Run on ASMR
freq_bands = {
    'Delta': (1, 4), 'Theta': (4, 8), 'Alpha': (8, 12), 'Beta': (12, 30), 'Gamma': (30, 45)
}
extracting_psd_features(
    "./control_epochs/",
    freq_bands,
    num_participants=None,
    num_seconds=10,
    sampling_rate=250
)

Directory: ./control_epochs/  Label: 0
Processing subject: 1-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  PSD array shape: (18, 31, 5)
  Saved PSD for 1-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 10-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  PSD array shape: (18, 31, 5)
  Saved PSD for 10-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 11-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  PSD array shape: (18, 31, 5)
  Saved PSD for 11-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 12-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  PSD array shape: (18, 31, 5)
  Saved PSD for 12-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 13-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  PSD array shape: (18, 31, 5)
  Saved PSD for 13-epochs-epo → samples=18, channels=31, freqs=5
Processing subject: 14-epochs-epo
labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  P

In [11]:
import numpy as np

data = np.load("/home/s.dharia-ra/Shyamal/major_revisions_ASMR/control_psd_features_new_MR/1-epochs-epo_psd.npz")
data.keys()

features = data["psd_open"]
features

array([[[1.05547164e-12, 6.36847049e-13, 2.68938621e-13, 3.22014264e-13,
         2.76912425e-13],
        [3.79467783e-13, 3.06340165e-13, 1.98557753e-13, 1.25608739e-13,
         6.84013829e-14],
        [1.98632517e-12, 1.30490538e-12, 5.53355034e-13, 1.51403083e-13,
         1.02315734e-13],
        ...,
        [4.65202447e-13, 2.61430280e-13, 1.27857315e-13, 7.18514664e-14,
         5.35122127e-14],
        [2.71945846e-12, 1.40599083e-12, 4.24664124e-13, 1.55330122e-13,
         9.61508348e-14],
        [7.93073075e-13, 5.23864148e-13, 1.96098910e-13, 1.52269324e-13,
         1.25880515e-13]],

       [[9.54418542e-13, 6.22746109e-13, 5.90705726e-13, 3.34019483e-13,
         3.04383994e-13],
        [4.37255370e-13, 3.27551855e-13, 1.79196420e-13, 1.15096732e-13,
         9.51584558e-14],
        [1.71195904e-12, 1.18228130e-12, 1.04597125e-12, 1.60997296e-13,
         1.03126358e-13],
        ...,
        [3.27178953e-13, 2.55922439e-13, 1.36866541e-13, 7.43606223e-14,
        

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Compute sex ratio and age statistics for ASMR and Control groups.
"""

import pandas as pd

# ─── raw data ──────────────────────────────────────────────────────────────────
records = [
    #  ID,     Sex, Age,  Group
    ("asmr1",  "F", 21,  "ASMR"),
    ("asmr2",  "M", 21,  "ASMR"),
    ("asmr3",  "F", 37,  "ASMR"),
    ("asmr4",  "M", 24,  "ASMR"),
    ("asmr5",  "F", 20,  "ASMR"),
    ("asmr6",  "M", 19,  "ASMR"),
    ("asmr7",  "M", 31,  "ASMR"),
    ("asmr8",  "F", 21,  "ASMR"),
    ("asmr9",  "F", 21,  "ASMR"),
    ("asmr10", "F", 22,  "ASMR"),
    ("asmr11", "F", 26,  "ASMR"),
    ("asmr12", "F", 27,  "ASMR"),
    ("asmr13", "F", 23,  "ASMR"),
    ("asmr14", "M", 23,  "ASMR"),
    ("asmr15", "F", 23,  "ASMR"),
    ("asmr16", "M", 24,  "ASMR"),
    ("ctrl1",  "F", 22,  "Control"),
    ("ctrl2",  "M", 21,  "Control"),
    ("ctrl3",  "F", 39,  "Control"),
    ("ctrl4",  "M", 23,  "Control"),
    ("ctrl5",  "F", 20,  "Control"),
    ("ctrl6",  "M", 20,  "Control"),
    ("ctrl7",  "M", 29,  "Control"),
    ("ctrl8",  "F", 22,  "Control"),
    ("ctrl9",  "F", 23,  "Control"),
    ("ctrl10", "F", 21,  "Control"),
    ("ctrl12", "F", 24,  "Control"),
    ("ctrl13", "F", 26,  "Control"),
    ("ctrl14", "M", 23,  "Control"),
    ("ctrl15", "F", 21,  "Control"),
    ("ctrl16", "M", 24,  "Control"),
]

# ─── load into DataFrame ───────────────────────────────────────────────────────
cols = ["ID", "Sex", "Age", "Group"]
df = pd.DataFrame(records, columns=cols)

# ─── helper to compute summary for a subset ────────────────────────────────────
def summarize(sub):
    n_f = (sub["Sex"] == "F").sum()
    n_m = (sub["Sex"] == "M").sum()
    f_m_ratio = f"{n_f}:{n_m}"
    mean_age = sub["Age"].mean()
    std_age  = sub["Age"].std(ddof=1)  # sample SD
    return pd.Series({
        "N":      len(sub),
        "Female": n_f,
        "Male":   n_m,
        # "F:M":    f_m_ratio,
        "Age µ":  round(mean_age, 2),
        "Age σ":  round(std_age, 2),
    })

# ─── group‑wise and overall summaries ─────────────────────────────────────────
summary = df.groupby("Group").apply(summarize)
summary.loc["Overall"] = summarize(df)

print(summary)


            N  Female  Male  Age µ  Age σ
Group                                    
ASMR     16.0    10.0   6.0  23.94   4.58
Control  15.0     9.0   6.0  23.87   4.81
Overall  31.0    19.0  12.0  23.90   4.61


/tmp/ipykernel_371372/3449420401.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = df.groupby("Group").apply(summarize)
